# FRB example

In [1]:
# imports
from importlib import reload
import os
from pkg_resources import resource_filename

import numpy as np

import pandas

from astropy.coordinates import SkyCoord
#from astropy.io import fits

from astropath import bayesian
from astropath import localization

# Dummy FRB (based on 180924)

## Coord

In [2]:
frb_coord = SkyCoord('21h44m25.255s -40d54m00.10s', frame='icrs')
frb_coord

<SkyCoord (ICRS): (ra, dec) in deg
    (326.10522917, -40.90002778)>

## Error ellipse

In [64]:
#eellipse = dict(a=0.3, b=0.1, theta=45.)
eellipse = dict(a=3, b=3, theta=90.)

#eellipse = dict(a=0.1, b=0.1, theta=0.)

## Localization dict

### See astropath/localization.py for the Data model

In [65]:
localiz = dict(type='eellipse', frb_coord=frb_coord, frb_eellipse=eellipse)

In [5]:
assert localization.vette_localization(localiz)

# Candidates

## Load

In [6]:
cand_file = os.path.join(resource_filename('astropath', 'data'), 'frb_example', 'frb180924_candidates.csv')

In [7]:
candidates = pandas.read_csv(cand_file, index_col=0)
candidates.head()

,ra,dec,half_light,VLT_FORS2_g
3,326.101738,-40.899771,0.458869,25.295416
7,326.105365,-40.900239,1.308629,21.319569
8,326.104186,-40.900180,0.814683,24.272838
9,326.106237,-40.899279,0.501247,25.474828


## Coords

In [8]:
c_coords = SkyCoord(ra=candidates.ra, dec=candidates.dec, unit='deg')
c_coords

<SkyCoord (ICRS): (ra, dec) in deg
    [(326.1017384 , -40.89977098), (326.10536507, -40.90023897),
     (326.10418611, -40.90018023), (326.10623701, -40.8992786 )]>

# Priors

## Guiding dict

In [9]:
offset_prior = dict(method='exp', 
                    max=6.,
                   ang_size=candidates.half_light.values)
priors = dict(offset=offset_prior, 
              O='inverse', 
              U=0., 
              name='Adopted')

## Calculate raw

In [10]:
# Raw priors
raw_prior_Oi = bayesian.raw_prior_Oi(priors['O'], 
                                     candidates.VLT_FORS2_g.values, 
                                     half_light=candidates.half_light.values)

In [11]:
candidates['P_O_raw'] = raw_prior_Oi
candidates

,ra,dec,half_light,VLT_FORS2_g,P_O_raw
3,326.101738,-40.899771,0.458869,25.295416,73.150261
7,326.105365,-40.900239,1.308629,21.319569,2017.091915
8,326.104186,-40.900180,0.814683,24.272838,157.983590
9,326.106237,-40.899279,0.501247,25.474828,64.277492


## Normalize

In [12]:
candidates['P_O'] = bayesian.renorm_priors(candidates.P_O_raw.values, priors['U'])

In [13]:
candidates

,ra,dec,half_light,VLT_FORS2_g,P_O_raw,P_O
3,326.101738,-40.899771,0.458869,25.295416,73.150261,0.031633
7,326.105365,-40.900239,1.308629,21.319569,2017.091915,0.872255
8,326.104186,-40.900180,0.814683,24.272838,157.983590,0.068317
9,326.106237,-40.899279,0.501247,25.474828,64.277492,0.027796


# Calculate $p(x|O)$

## Original

In [26]:
frb_coord.ra.deg

326.1052291666667

In [52]:
%%time
reload(bayesian)
p_xOi = bayesian.px_Oi(30.,  # box radius for grid, in arcsec
                       frb_coord,
                       eellipse,
                       c_coords,
                       priors['offset'], step_size=0.02)
#p_xOi

> /data/Projects/FRB_Software/astropath/astropath/bayesian.py(262)px_Oi()
    260     import pdb; pdb.set_trace()
    261 
--> 262     p_xOis, grids = [], []
    263     # TODO -- multiprocess this
    264     for icand, cand_coord in enumerate(cand_coords):



ipdb>  np.sum(L_wx)


2498.333611111112


ipdb>  np.sum(L_wx)*0.02**2


0.9993334444444448


ipdb>  exit


BdbQuit: 

In [ ]:
%%time
reload(localization)
reload(bayesian)
#
p_xOi = bayesian.px_Oi_fixed(30.,  # box radius for grid, in arcsec
                       localiz,
                       c_coords,
                       priors['offset'], step_size=0.05)
#p_xOi

Grid spacing = 0.050041701417846696
> /data/Projects/FRB_Software/astropath/astropath/bayesian.py(172)px_Oi_fixed()
    170     dec = localiz['frb_coord'].dec.deg + ycoord/3600.
    171     import pdb; pdb.set_trace()
--> 172     L_wx = localization.calc_LWx(ra, dec, localiz)
    173 
    174     p_xOis, grids = [], []



ipdb>  xcoord[0,0]


-30.0


ipdb>  localiz['frb_coord'].ra.deg 


326.1052291666667


...

ipdb>  ra[0,0] - localiz['frb_coord'].ra.deg 


-0.008333333333325754


ipdb>  3600*(ra[0,0] - localiz['frb_coord'].ra.deg)


-29.999999999972715


ipdb>  exit


In [21]:
candidates['p_xO'] = p_xOi

In [22]:
candidates

,ra,dec,half_light,VLT_FORS2_g,P_O_raw,P_O,p_xO
3,326.101738,-40.899771,0.458869,25.295416,73.150261,0.031633,0.992440
7,326.105365,-40.900239,1.308629,21.319569,2017.091915,0.872255,0.130361
8,326.104186,-40.900180,0.814683,24.272838,157.983590,0.068317,0.318530
9,326.106237,-40.899279,0.501247,25.474828,64.277492,0.027796,0.836039


# $p(x)$

In [14]:
p_x = np.sum(candidates.P_O * candidates.p_xO)
p_x

4.1327985693607774e-06

# Posteriors $P(O|x)$

In [15]:
P_Oix = candidates.P_O * p_xOi / p_x
candidates['P_Ox'] = P_Oix

In [16]:
candidates

,ra,dec,half_light,VLT_FORS2_g,P_O_raw,P_O,p_xO,P_Ox
3,326.101738,-40.899771,0.458869,25.295416,73.150261,0.031633,0.000000e+00,0.000000e+00
7,326.105365,-40.900239,1.308629,21.319569,2017.091915,0.872255,4.564015e-06,9.632657e-01
8,326.104186,-40.900180,0.814683,24.272838,157.983590,0.068317,2.222214e-06,3.673426e-02
9,326.106237,-40.899279,0.501247,25.474828,64.277492,0.027796,3.099763e-23,2.084783e-19
